In [1]:
# DS4200
# Final Project
# Data Cleaning Portion

In [117]:
import pandas as pd
import numpy as np

In [3]:
# Read in the files
resort_df = pd.read_csv("H1.csv")
city_df = pd.read_csv("H2.csv")

In [109]:
# Append the 2 datasets
# Create a column to differentiate the booking type (city, hotel)
resort_df["HotelType"] = "Resort"
city_df["HotelType"] = "City"

# Now conduct the merge
bookings_df = pd.concat((resort_df, city_df), axis=0)

# Filter the data down to 3 countries --> Spain, France, Italy
#country_lst = list("ESP", "FRA", "ITA")
bookings_df = bookings_df.loc[bookings_df["Country"].isin(["ESP", "FRA", "ITA"])]
print(set(bookings_df["Country"]))

{'ITA', 'FRA', 'ESP'}


In [143]:
# Remove whitespace to find NULL values
# and convert them to represent the NaN data type
bookings_df = bookings_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
bookings_df.replace("NULL", np.nan, inplace=True)

# Find the amount of NaN rows in the dataset 
# and remove them
null_rows = bookings_df[bookings_df.isnull().all(axis=1)]
#print(null_rows)  #32

null_idx = null_rows.index.to_list()
bookings_df = bookings_df.drop(index=null_idx)

# Find which column's have the highest prevalency
# of null values
for col in list(bookings_df.columns):
    null_amt = bookings_df[col].isnull().sum()
    pct = (null_amt / len(bookings_df)) * 100
    print(f"{col}: {round(pct, 3)}%")

IsCanceled: 0.0%
LeadTime: 0.0%
ArrivalDateYear: 0.0%
ArrivalDateMonth: 0.0%
ArrivalDateWeekNumber: 0.0%
ArrivalDateDayOfMonth: 0.0%
StaysInWeekendNights: 0.0%
StaysInWeekNights: 0.0%
Adults: 0.0%
Children: 0.0%
Babies: 0.0%
Meal: 0.0%
Country: 0.0%
MarketSegment: 0.0%
DistributionChannel: 0.0%
IsRepeatedGuest: 0.0%
PreviousCancellations: 0.0%
PreviousBookingsNotCanceled: 0.0%
ReservedRoomType: 0.0%
AssignedRoomType: 0.0%
BookingChanges: 0.0%
DepositType: 0.0%
Agent: 6.752%
DaysInWaitingList: 0.0%
CustomerType: 0.0%
ADR: 0.0%
RequiredCarParkingSpaces: 0.0%
TotalOfSpecialRequests: 0.0%
ReservationStatus: 0.0%
ReservationStatusDate: 0.0%
HotelType: 0.0%


In [181]:
# Convert columns to Strings
string_cols = ["ArrivalDateMonth", "Meal", "Country", "MarketSegment",
               "DistributionChannel", "ReservedRoomType", "AssignedRoomType",
               "DepositType", "Agent", "CustomerType", "ReservationStatus", "HotelType"]
for col in string_cols:
    bookings_df[col] = bookings_df[col].astype(str)

# Convert column to Datetime objects
bookings_df["ReservationStatusDate"] = pd.to_datetime(bookings_df["ReservationStatusDate"])

In [187]:
# Based on the output above, it looks like the
# `Company` column is useless due to the high 
# percentage of NaN values
#bookings_df = bookings_df.drop("Company", axis=1)

# Replace NaN values in the `Agent` column
# with the most common
bookings_df["Agent"] = bookings_df["Agent"].replace("nan", np.nan)

mode_agent = bookings_df["Agent"].mode().iloc[0]
bookings_df["Agent"] = bookings_df["Agent"].fillna(mode_agent)